In [1]:
import sys
import os
import time,re
import numpy as np
from os import environ, listdir, makedirs
from os.path import dirname, exists, expanduser, isdir, join, splitext
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import nltk 
import string
import pandas as pd
from nltk.corpus import stopwords
from senticnet.senticnet import Senticnet
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk import sent_tokenize, word_tokenize, pos_tag
from random import uniform

In [2]:
def microtext(tweet):
    tweet = re.sub(r'(\$+)([a-zA-Z]+[.]?[a-zA-Z]*)', r'\2', tweet) #$AAPL -> aapl company
    tweet = re.sub(r'&#39;', r"'", tweet)
    tweet = re.sub(r'&amp;', r'&', tweet)
    tweet = re.sub(r'&quot;', r'', tweet)
    tweet = re.sub(r'(?:[\.]{2,10})', ' ', tweet) #delete suspension points
    tweet = re.sub(r'(?:[\!]{2,10})', '!', tweet) #delete repetition of !
    tweet = re.sub(r'(?:[\?]{2,10})', '?', tweet) #delete repetition of ?
    tweet = re.sub(r'((?:(?<!\S)\.+\d+-?)+(\d+)?)', r'0\1', tweet) #fix decimals .88 -> 0.88
    tweet = re.sub(r'[w]{3,10}[\.]+[\w]+[\.]?[a-z]*', '', tweet) #delete websites with no http
    tweet = re.sub(r'((?<!\S))(?:IP)([0-9xX]?)((?!\S)|[,.\\-_])', r'iphone\2', tweet)
    tweet = re.sub(r'((?<!\S))(?:ip)([0-9xX]?)((?!\S)|[,.\\-_])', r'iphone\2', tweet)
    tweet = re.sub(r'((?<!\S))(?:Ip)([0-9xX]?)((?!\S)|[,.\\-_])', r'iphone\2', tweet)
    tweet = re.sub(r'((?<!\S))(?:iP)([0-9xX]?)((?!\S)|[,.\\-_])', r'iphone\2', tweet)
    tweet = re.sub(r'((btfd)|(BTFD)|(Btfd))', 'buy the failed dip', tweet)
    tweet = re.sub(r'((btd)|(BTD)|(Btd))', 'buy the dip', tweet) 
    tweet = re.sub(r'((h8t)|(H8T)|(H8t))', 'hate', tweet)
    tweet = re.sub(r'((btmfd)|(BTMFD)|(Btmfd))', 'burn this mother fucker down', tweet)
    tweet = re.sub(r'((wtf)|(WTF)|(Wtf))', 'what a fuck', tweet)
    tweet = re.sub(r'((lmao)|(LMAO)|(Lmao))', 'laughing my ass off', tweet)
    tweet = re.sub(r'((lmfao)|(LMFAO)|(Lmfao))', 'laughing my fucking ass off', tweet)
    tweet = re.sub(r'((imho)|(IMHO)|(Imho))', 'in my humble opinion', tweet)
    tweet = re.sub(r'((plz)|(PLZ)|(Plz))', 'please', tweet)
    tweet = re.sub(r'((govt)|(GOVT)|(Govt))', 'government', tweet)
    tweet = re.sub(r'(?<!\S)[n](?!\S)', 'and', tweet)
    tweet = re.sub(r'(?<!\S)([U]|[u])(?!\S)', 'you', tweet)
    tweet = re.sub(r'(?<!\S)[w][\/]?(?!\S)', 'with', tweet)
    tweet = re.sub(r'(?<!\S)[b](?!\S)', 'be', tweet) #but b could means also buy
    tweet = re.sub(r'(?<!\S)[t](?!\S)', 'the', tweet)
    tweet = re.sub(r'(?<!\S)(aint)(?!\S)', 'is not', tweet)
    tweet = re.sub(r'(?<!\S)(rez)(?!\S)', 'reservation', tweet)
    tweet = re.sub(r'(?<!\S)((sep)|(SEP)|(sept)|(SEPT)|(Sept))(?!\S)', 'september', tweet)
    tweet = re.sub(r'(?<!\S)((jan)|(JAN)|(jann)|(JANN)|(Jan))(?!\S)', 'january', tweet)
    tweet = re.sub(r'(?<!\S)((feb)|(FEB)|(febr)|(FEBR)|(Feb))(?!\S)', 'february', tweet)
    tweet = re.sub(r'(?<!\S)((mar)|(MAR)|(Mar))(?!\S)', 'march', tweet)
    tweet = re.sub(r'(?<!\S)((apr)|(APR)|(Apr))(?!\S)', 'april', tweet)
    tweet = re.sub(r'(?<!\S)((aug)|(AUG)|(Aug))(?!\S)', 'august', tweet)
    tweet = re.sub(r'(?<!\S)((oct)|(OCT)|(Oct))(?!\S)', 'october', tweet)
    tweet = re.sub(r'(?<!\S)((nov)|(NOV)|(Nov))(?!\S)', 'november', tweet)
    tweet = re.sub(r'(?<!\S)((dec)|(DEC)|(Dec))(?!\S)', 'december', tweet)
    tweet = re.sub(r'(?<!\S)((HOD)|(hod)|(Hod))(?!\S)?', 'head of department', tweet)
    tweet = re.sub(r'(?<!\S)((BS)|(bs)|(Bs))(?!\S)', 'bullshit', tweet)
    tweet = re.sub(r'(?<!\S)((ws)|(WS)|(Ws))(?!\S)', 'wall street', tweet)
    tweet = re.sub(r'((AAPL)|(aapl)|(Aapl))', r'apple', tweet)
    tweet = re.sub(r'((QQQ)|(qqq)|(Qqq))', r'NASDAQ 100 index tracking stock', tweet)
    tweet = re.sub(r'((CRM)|(crm)|(Crm))', r'salesforce', tweet)
    tweet = re.sub(r'((IMNP)|(imnp)|(Imnp))', r'immune pharmaceuticals', tweet)
    tweet = re.sub(r'(?<!\S)((DRYS)|(drys)|(Drys))(?!\S)', r'dryships', tweet)
    tweet = re.sub(r'(?<!\S)((TOPS)|(tops)|(Tops))(?!\S)', r'top ships', tweet)
    tweet = re.sub(r'((DCTH)|(dcth)|(Dtch))', r'delcath systems', tweet)
    tweet = re.sub(r'((AVEO)|(aveo)|(Aveo))', r'aveo pharmaceuticals', tweet)
    tweet = re.sub(r'((OPXAW)|(opxaw)|(Opxaw))', r'opexa therapeutics', tweet)
    tweet = re.sub(r'((GRPN)|(grpn)|(Grpn))', r'groupon', tweet)
    tweet = re.sub(r'((MSFT)|(msft)|(Msft))', r'microsoft', tweet)
    tweet = re.sub(r'((AMLP)|(amlp)|(Amlp))', r'alerian mlp', tweet)
    tweet = re.sub(r'((TSLA)|(tsla)|(Tsla))', r'tesla', tweet)
    tweet = re.sub(r'(?<!\S)((BABA)|(baba)|(Baba))(?!\S)', r'alibaba', tweet)
    tweet = re.sub(r'(?<!\S)((VIX)|(vix)|(Vix))(?!\S)', r'volatility index', tweet)
    tweet = re.sub(r'(?<!\S)((SPY)|(spy)|(Spy))(?!\S)', r'spdr s&p 500', tweet)
    tweet = re.sub(r'((NVDA)|(nvda)|(Nvda))', r'nvidia', tweet)
    tweet = re.sub(r'((GOOG)|(goog)|(Goog)|(GOOGL)|(googl)|(Googl))(?!\S)', r'google', tweet)
    tweet = re.sub(r'((FB)|(fb)|(Fb))', r'facebook', tweet)
    tweet = re.sub(r'((NFLX)|(nflx)|(Nflx))', r'netflix', tweet)
    tweet = re.sub(r'(SHOP)(?!\S)', r'shopify', tweet)
    tweet = re.sub(r'((BIIB)|(biib)|(Biib))', r'biogen idec', tweet)
    tweet = re.sub(r'((LRCX)|(lrcx)|(Lrcx))', r'lam research', tweet)
    tweet = re.sub(r'((AVGO)|(avgo)|(Avgo))', r'avago technologies', tweet)
    tweet = re.sub(r'((XLF)|(xlf)|(Xlf))', r'financial select sector spdr', tweet)
    tweet = re.sub(r'((TLT)|(tlt)|(Tlt))', r'ishares lehman 20 year', tweet)
    tweet = re.sub(r'((IWM)|(iwm)|(Iwm))', r'ishares russel 2000', tweet)
    tweet = re.sub(r'(?<!\S)((GS)|(gs)|(Gs))(?!\S)', r'goldman sachs', tweet)
    tweet = re.sub(r'(?<!\S)((BAC)|(bac))(?!\S)', r'bank of america', tweet)
    tweet = re.sub(r'(?<!\S)((CAT)|(cat)|(Cat))(?!\S)', r'caterpillar', tweet)
    tweet = re.sub(r'(?<!\S)((FL)|(fl)|(Fl))(?!\S)', r'foot locker', tweet)
    tweet = re.sub(r'((WMT)|(wmt)|(Wmt))', r'walmart', tweet)
    tweet = re.sub(r'(?<!\S)((CRUS)|(crus)|(Crus))(?!\S)', r'cirrus logic', tweet)
    return (tweet)

In [3]:
#L&m on finance
with open('/Users/FINANCE_DOMAIN.txt') as nuovo:
    result = list(list(l for l in e.split("','") if l) for e in nuovo.read().split("\n"))
result = [x for x in result if not x == []]
result = [x for x in result if not "None'" in x]
#remove duplicate
df = pd.DataFrame(result)
df = df.drop_duplicates()
result = df.values.tolist()
#substitute bearish with negative and bullish with positive
for i in range(len(result)):
    if "{'basic': 'Bullish'}'" in result[i][3]:
         result[i][3] = "positive"
    else:
        result[i][3] = "negative"

emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
regex_str = [
    emoticons_str,
    r'(?:[\.]{2,10})', #suspension points
    r'(?:[Ss]+[&]+[Pp]+)',
    r'\\(?:[a-z0-9]{3,3}[\\])+[a-z0-9]{3,3}', #unicode
    r"(?:\$+[\w_]*[.]?[\d]*)", #jargons
    #r'\\n\\n',#enter and next line
    r'\\n',
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-.]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
    r'(?:(?:\.?\d+,?-?%?\$?)+(?:\.?\/?\d+\/?\d*)?(?:[cb])?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]

tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)

def tokenize(s):
    return tokens_re.findall(s)

def preprocess(s, lowercase=True):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens

punctuation = ['"', '#', '$', '%', '(', ')', '*', '-', ',', '/', '<', '>', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~']
stop = punctuation + ['\\n', 'etc']
list_LM = []
lemmas=set()
POSITIVENESS = []
NEGATIVENESS = []
with open('/Users/LM/Whole.txt','r') as whole, open('/Users/LM/Positive.txt','r') as pos, open('/Users/LM/Negative.txt','r') as neg:        
    for line in whole:
        words = [term for term in preprocess(line) if term not in stop]
        for w in words:
            list_LM.append(w)
    for line in pos:
        words = [term for term in preprocess(line) if term not in stop] 
        for w in words:
            POSITIVENESS.append(w)
    for line in neg:
        words = [term for term in preprocess(line) if term not in stop]
        for w in words:
            NEGATIVENESS.append(w)


new_sentiment = {}
train_data = []
train_labels = []
test_data = []
test_labels = []
list_corpus = []

lemmatizer = WordNetLemmatizer()
def penn_to_wn(tag):
    
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return None

def add_words(lemma):
    count_posi = 0
    count_nega = 0        
    for i in range(len(result)):
        ID = result[i][0]
        tweet = result[i][2]
        label = result[i][3]
        tweet = microtext(tweet)
        terms_stop = [term for term in preprocess(tweet) if term not in stop and not term.startswith(('http', '\\x', '#'))]
        tagged_sentence = pos_tag(terms_stop)
        for word, tag in tagged_sentence:
            wn_tag = penn_to_wn(tag)
            if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV, wn.VERB):
                continue

            lemmat = lemmatizer.lemmatize(word, pos=wn_tag)
            if not lemmat:
                continue
            if lemma == lemmat:
                if label == 'positive':
                    count_posi += 1
                else:
                    count_nega += 1
            else:
                continue
    unbalance_ratio = 5
    if (count_posi > (count_nega*unbalance_ratio)) and (count_posi > unbalance_ratio):
        new_sentiment[lemma] = 0.5
        lemmas.add(lemma)
    elif ((count_posi*unbalance_ratio) < count_nega) and (count_nega > unbalance_ratio):
        new_sentiment[lemma] = -0.5
        lemmas.add(lemma)


#compute the percentage of right prediction of tweets that have the specific word. The word here has the score not updated
def baseline_performance(baseline_word):
    right_1 = 0
    wrong_1 = 0
    for lll in range(len(test_data)):
        check_word_in_sentence = []
        tagged_sentence = pos_tag(test_data[lll])
        for word, tag in tagged_sentence:
            wn_tag = penn_to_wn(tag)
            if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV, wn.VERB):
                continue
            lemma = lemmatizer.lemmatize(word, pos=wn_tag)
            if not lemma:
                continue
            check_word_in_sentence.append(lemma)
        if baseline_word in check_word_in_sentence:
            compare = test_labels[lll]
            baseline_word_score = np.zeros(VOCABULARY_SIZE)
            for yyy in check_word_in_sentence:
                if yyy in list_new:
                    sentiment = 0.0
                    if (yyy == baseline_word) and (baseline_word in CHANGE_IN_THE_LEXICON):
                        sentiment = CHANGE_IN_THE_LEXICON[baseline_word] 
                    else:
                        if yyy in new_sentiment.keys():
                            sentiment = new_sentiment[yyy]
                        else:
                            if yyy in POSITIVENESS:
                                sentiment = 1
                            elif yyy in NEGATIVENESS:
                                sentiment = -1
                    if yyy == 'not':
                        sentiment = 0.0
                    indexes = word_index[yyy]
                    baseline_word_score[indexes] = sentiment
                else:
                    continue
            prova = []
            prova.append(baseline_word_score)
            prediction_liblinear = classifier_liblinear.predict(prova)
            if 'not' in check_word_in_sentence:
                if prediction_liblinear == 'positive':
                    prediction_liblinear = 'negative'
                else:
                    prediction_liblinear = 'positive'
                    
            if compare == prediction_liblinear:
                right_1 += 1
            else:
                wrong_1 += 1    
        else:
            continue
    baseline_performance.performance_baseline = (right_1/(right_1+wrong_1))
    return None
                
def choose_the_best_arm(position):
    already_modified = {}
    for j in range(len(position)):
        baseline_word = list_new[position[j]]
        already_modified[baseline_word] = False
        if baseline_word in CHANGE_IN_THE_LEXICON.keys():
            #narrow the interval and the number of max interations cosi tengo conto (parzialmente anche di tutte gli altri cambiamenti che sto facendo => considere le interdipendenze tra le parole)
            already_modified[baseline_word] = True
            continue
        else:
            print(list_new[position[j]], ' try to change its polarity from PS = ', test_word_score[position[j]], 'to: ')
            for h in range(0,10):
                old_score = test_word_score[position[j]]
                new_score = old_score + uniform(-1,1)
                performance_baseline = 0.0
                baseline_performance(baseline_word)
                print("PS^ = ", new_score)
                right = 0 
                wrong = 0
                performance = 0.0
                for ll in range(len(test_data)):
                    check_word_in_sentence = []
                    tagged_sentence = pos_tag(test_data[ll])
                    for word, tag in tagged_sentence:
                        wn_tag = penn_to_wn(tag)
                        if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV, wn.VERB):
                            continue
                        lemma = lemmatizer.lemmatize(word, pos=wn_tag)
                        if not lemma:
                            continue
                        check_word_in_sentence.append(lemma)
                    if list_new[position[j]] in check_word_in_sentence:
                            #print('ce un altro errore')
                        compare = test_labels[ll]
                        new_test_word_score = np.zeros(VOCABULARY_SIZE)
                        for yy in check_word_in_sentence:
                            if yy in list_new:
                                sentiment = 0.0
                                if yy == list_new[position[j]]:
                                    sentiment = new_score
                                else:
                                    
                                    if yy in new_sentiment.keys():
                                        sentiment = new_sentiment[yy]
                                    else:
                                        if yy in POSITIVENESS:
                                            sentiment = 1
                                        elif yy in NEGATIVENESS:
                                            sentiment = -1
                                if yy == 'not':
                                    sentiment = 0.0
                                indexes = word_index[yy]
                                new_test_word_score[indexes] = sentiment
                            else:
                                continue
                        prova = []
                        prova.append(new_test_word_score)
                        prediction_liblinear = classifier_liblinear.predict(prova)
                        if 'not' in check_word_in_sentence:
                            if prediction_liblinear == 'positive':
                                prediction_liblinear = 'negative'
                            else:
                                prediction_liblinear = 'positive'
                        if compare == prediction_liblinear:
                            right += 1
                        else:
                            wrong += 1    
                    else:
                        continue
                performance = (right/(right+wrong))
            #print('new perf %f' % (performance))
                    #check if there is an improvement
                if performance > baseline_performance.performance_baseline:
                    print("PS' = ", new_score)
                    word_with_new_score = list_new[position[j]]
                    #if performance < 0.6:
                        #riprova[word_with_new_score] = performance
                        #print(word_with_new_score, ' aggiunto alla lista dei cattivi')
                    ALREADY_MODIFIED[word_with_new_score] = 1
                    CHANGE_IN_THE_LEXICON[word_with_new_score] = new_score #cosi cosnidero i cambiamenti solo da un tweet all'altro e non per ogni parola
                    riprova_tutto[word_with_new_score] = performance
                    break
                else:
                    continue
    for g in already_modified:
        if already_modified[g] == False:
            continue
        else:
            ALREADY_MODIFIED[g] += 1
            if ALREADY_MODIFIED[g] > 9:
                continue
            else:
                print(g, " try to change its polarity from PS' = ", CHANGE_IN_THE_LEXICON[g], 'to: ')
                for w in range(0,10-ALREADY_MODIFIED[g]):
                    old_score = CHANGE_IN_THE_LEXICON[g]
                    new_score = old_score + uniform(-1/ALREADY_MODIFIED[g],1/ALREADY_MODIFIED[g])
                    print("PS^^ = ", new_score)
                    performance_baseline = 0.0
                    baseline_word = g
                    baseline_performance(baseline_word)
                    right = 0 
                    wrong = 0
                    performance = 0.0
                        #change the score of the word
                    for ll in range(len(test_data)):
                        check_word_in_sentence = []
                        tagged_sentence = pos_tag(test_data[ll])
                        for word, tag in tagged_sentence:
                            wn_tag = penn_to_wn(tag)
                            if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV, wn.VERB):
                                continue
                            lemma = lemmatizer.lemmatize(word, pos=wn_tag)
                            if not lemma:
                                continue
                            check_word_in_sentence.append(lemma)
                        if g in check_word_in_sentence:
                            compare = test_labels[ll]
                            new_test_word_score = np.zeros(VOCABULARY_SIZE)
                            for yy in check_word_in_sentence:
                                if yy in list_new:
                                    sentiment = 0.0
                                    if yy == g:
                                        sentiment = new_score
                                    else:
                                        
                                        if yy in new_sentiment.keys():
                                            sentiment = new_sentiment[yy]
                                        else:
                                            if yy in POSITIVENESS:
                                                sentiment = 1
                                            elif yy in NEGATIVENESS:
                                                sentiment = -1
                                    if yy == 'not':
                                        sentiment = 0.0
                                    indexes = word_index[yy]
                                    new_test_word_score[indexes] = sentiment
                                else:
                                    continue
                            prova = []
                            prova.append(new_test_word_score)
                            prediction_liblinear = classifier_liblinear.predict(prova)
                            if 'not' in check_word_in_sentence:
                                if prediction_liblinear == 'positive':
                                    prediction_liblinear = 'negative'
                                else:
                                    prediction_liblinear = 'positive'
                            if compare == prediction_liblinear:
                                right += 1
                            else:
                                wrong += 1    
                        else:
                            continue
                    performance = (right/(right+wrong))
            #print('new perf %f' % (performance))
                    #check if there is an improvement
                    if performance > baseline_performance.performance_baseline:
                        word_with_new_score = g
                        print("PS'' = ", new_score)
                        #if performance < 0.6:
                            #riprova[word_with_new_score] = performance
                            #print(word_with_new_score, ' aggiunto alla lista dei cattivi')
                        #if  (old_score > 0.0 and new_score < 0.0) or (old_score < 0.0 and new_score > 0.0):
                            #riprova[word_with_new_score] = performance
                            #print(word_with_new_score, ' aggiunto alla lista dei cattivi')
                        CHANGE_IN_THE_LEXICON[word_with_new_score] = new_score 
                        #per_frank[(word_with_new_score,old_score)] = new_score
                        riprova_tutto[word_with_new_score] = performance
                        break
                    else:
                        continue
                        
    return None

In [ ]:
vector_just_apple = 0
for i in range(len(result)):
    print(str(i)+'th record processed')
    ID = result[i][0]
    tweet = result[i][2]
    label = result[i][3]
    tweet = microtext(tweet)
    terms_stop = [term for term in preprocess(tweet) if term not in stop and not term.startswith(('http', '\\x', '#'))]
    tagged_sentence = pos_tag(terms_stop)
    sentence_lemmas = []
    for word, tag in tagged_sentence:
        wn_tag = penn_to_wn(tag)
        if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV, wn.VERB):
            continue

        lemma = lemmatizer.lemmatize(word, pos=wn_tag)
        if not lemma:
            continue
        list_corpus.append(lemma)
        sentence_lemmas.append(lemma)
        
    corpus = set(list_corpus)
    set_lemmas = set(sentence_lemmas)
    print('lemmatized')
    print(set_lemmas.intersection(lemmas))
    #add words to sent lex
    if (set_lemmas & lemmas) and (set_lemmas.intersection(lemmas) != {'apple'}):
        continue
    else: 
        vector_just_apple +=1
        for word, tag in tagged_sentence:
            wn_tag = penn_to_wn(tag)
            if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV, wn.VERB):
                continue
            lemma = lemmatizer.lemmatize(word, pos=wn_tag)
            if not lemma:
                continue
            #assign polarity
            if (wn_tag in (wn.VERB, wn.ADJ)) and (lemma not in lemmas):
                print('prepare to add word')
                add_words(lemma)
                print('added word')
            else:
                continue
print('vectors with just apple: ', vector_just_apple)
print('lexicon extension: ', new_sentiment)
lemmas = set(list_LM)
new = lemmas.intersection(corpus)
list_new = list(new) 
word_index = {w: idx for idx, w in enumerate(list_new)}  
VOCABULARY_SIZE = len(list_new)  
print('VOCABULARY_SIZE %f' % (VOCABULARY_SIZE))

In [ ]:
count_pos = 0
count_neg = 0    
real_train = []
real_test = []
#split test and training data
for n in range(len(result)):
    ID = result[n][0]
    tweet = result[n][2]
    label = result[n][3]
    tweet = microtext(tweet)
    terms_stop = [term for term in preprocess(tweet) if term not in stop and not term.startswith(('http', '\\x', '#'))]
    if label == 'positive':
        count_pos += 1
        if count_pos < 1501:
            test_data.append(terms_stop)
            test_labels.append(label)
        else:
            train_data.append(terms_stop)
            train_labels.append(label)    
    else:
        count_neg += 1
        if count_neg < 1501:
            test_data.append(terms_stop)
            test_labels.append(label)
        else:
            train_data.append(terms_stop)
            train_labels.append(label)
        
#create input for classification        
for j in range(len(train_data)):
    train_word_score = np.zeros(VOCABULARY_SIZE)
    tagged_sentence = pos_tag(train_data[j])
    for word, tag in tagged_sentence:
        wn_tag = penn_to_wn(tag)
        if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV, wn.VERB):
            continue

        lemma = lemmatizer.lemmatize(word, pos=wn_tag)
        if not lemma:
            continue

        if lemma in list_new: 
        # Take the first sense, the most common
            sentiment = 0.0   
            if lemma in new_sentiment.keys():
                sentiment = new_sentiment[lemma]
            else:
                if lemma in POSITIVENESS:
                    sentiment = 1
                elif lemma in NEGATIVENESS:
                    sentiment = -1
            if lemma == 'not':
                sentiment = 0.0
            indexes = word_index[lemma]
            train_word_score[indexes] = sentiment
        else:
            continue
        
    real_train.append(train_word_score)
    
contiamo = 0
CHANGE_IN_THE_LEXICON = {}
per_frank = {}
ALREADY_MODIFIED = {}
riprova = {}
riprova_tutto = {}
for l in range(len(test_data)): #len(test_data)
    controlla_not = []
    compare = test_labels[l]
    test_word_score = np.zeros(VOCABULARY_SIZE)
    temp_test_word_score = np.zeros(VOCABULARY_SIZE)
    tagged_sentence = pos_tag(test_data[l])
    for word, tag in tagged_sentence:
        wn_tag = penn_to_wn(tag)
        if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV, wn.VERB):
            continue

        lemma = lemmatizer.lemmatize(word, pos=wn_tag)
        if not lemma:
            continue
            
        controlla_not.append(lemma)

        if lemma in list_new: 
        # Take the first sense, the most common
            sentiment = 0.0  
            if lemma in new_sentiment.keys():
                sentiment = new_sentiment[lemma]                 
            else:
                if lemma in POSITIVENESS:
                    sentiment = 1
                elif lemma in NEGATIVENES:
                    sentiment = -1
            if lemma == 'not': 
                sentiment = 0.0
            indexes = word_index[lemma]
            test_word_score[indexes] = sentiment
            temp_test_word_score[indexes] = sentiment
            #new_test_word_score[indexes] = sentiment
        else:
            continue
    prova = []
    prova.append(test_word_score)
    list_rewards = []
    classifier_liblinear = svm.LinearSVC()
    classifier_liblinear.fit(real_train, train_labels)
    prediction_liblinear = classifier_liblinear.predict(prova)
    if 'not' in controlla_not:
        if prediction_liblinear == 'positive':
            prediction_liblinear = 'negative'
        else:
            prediction_liblinear = 'positive'
    if compare == prediction_liblinear:
        continue
    else:
        contiamo +=1
        position = []
        for q in range(len(test_word_score)):
            if test_word_score[q] != 0:
                position.append(q)
            else:
                continue
        choose_the_best_arm(position)
        
print('vector explored: ', contiamo)
print('after convergence: ', CHANGE_IN_THE_LEXICON)

In [ ]:
#just convergence
final_real_train = []
final_real_test = []        
#train again SVM with new scores given by the updated sentiment lexicon final_train_word_score
for j in range(len(train_data)):
    controlla_not = []
    final_train_word_score = np.zeros(VOCABULARY_SIZE)
    tagged_sentence = pos_tag(train_data[j])
    for word, tag in tagged_sentence:
        wn_tag = penn_to_wn(tag)
        if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV, wn.VERB):
            continue

        lemma = lemmatizer.lemmatize(word, pos=wn_tag)
        if not lemma:
            continue

        if lemma in list_new: 
        # Take the first sense, the most common
            sentiment = 0.0   
            if lemma in new_sentiment.keys():
                sentiment = new_sentiment[lemma]
            else:
                if lemma in POSITIVENESS:
                    sentiment = 1
                elif lemma in NEGATIVENESS:
                    sentiment = -1
            if lemma in CHANGE_IN_THE_LEXICON.keys():
                sentiment = CHANGE_IN_THE_LEXICON[lemma]
            if lemma == 'not':
                sentiment = 0.0
            indexes = word_index[lemma]
            final_train_word_score[indexes] = sentiment
            #temp.append(sentiment)
        else:
            continue
        #if all(p == 0.0 for p in temp):
            #print(1)
        #else: 
            #continue
        
    final_real_train.append(final_train_word_score)

for l in range(len(test_data)):      
    final_test_word_score = np.zeros(VOCABULARY_SIZE)
    #test_word_score = np.zeros(VOCABULARY_SIZE)
    tagged_sentence = pos_tag(test_data[l])
    for word, tag in tagged_sentence:
        wn_tag = penn_to_wn(tag)
        if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV, wn.VERB):
            continue

        lemma = lemmatizer.lemmatize(word, pos=wn_tag)
        if not lemma:
            continue

        if lemma in list_new: 
        # Take the first sense, the most common
            sentiment = 0.0   
            if lemma in new_sentiment.keys():
                sentiment = new_sentiment[lemma]
            else:
                if lemma in POSITIVENESS:
                    sentiment = 1
                elif lemma in NEGATIVENESS:
                    sentiment = -1
            if lemma in CHANGE_IN_THE_LEXICON.keys():
                sentiment = CHANGE_IN_THE_LEXICON[lemma]
            if lemma == 'not':
                sentiment = 0.0
            indexes = word_index[lemma]
            final_test_word_score[indexes] = sentiment             
        else:
            continue
    final_real_test.append(final_test_word_score)
    

# Perform classification with SVM, kernel=linear
new_classifier_liblinear = svm.LinearSVC()
t0 = time.time()
new_classifier_liblinear.fit(final_real_train, train_labels)
t1 = time.time()
ok = 0.000
no = 0.000
for k in range(len(test_data)):
    controlla_not = []
    tagged_sentence = pos_tag(test_data[k])
    for word, tag in tagged_sentence:
        wn_tag = penn_to_wn(tag)
        if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV, wn.VERB):
            continue

        lemma = lemmatizer.lemmatize(word, pos=wn_tag)
        if not lemma:
            continue
        controlla_not.append(lemma)
        
    prediction = new_classifier_liblinear.predict(final_real_test[k])
    if 'not' in controlla_not:
        if prediction == 'positive':
            prediction = 'negative'
        else:
            prediction = 'positive'
            
    if prediction == test_labels[k]:
        ok += 1
    else:
        no += 1
accuracy = (ok/no)*100
print('accuracy: ', accuracy)

#NOW CHECK WITH REINFORCEMENT
updated_real_train = []
for j in range(len(train_data)):
    updated_train_word_score = np.zeros(VOCABULARY_SIZE)
    tagged_sentence = pos_tag(train_data[j])
    for word, tag in tagged_sentence:
        wn_tag = penn_to_wn(tag)
        if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV, wn.VERB):
            continue

        lemma = lemmatizer.lemmatize(word, pos=wn_tag)
        if not lemma:
            continue

        if lemma in list_new: 
        # Take the first sense, the most common
            sentiment = 0.0   
            if lemma in new_sentiment.keys():
                sentiment = new_sentiment[lemma]
            else:
                if lemma in POSITIVENESS:
                    sentiment = 1
                elif lemma in NEGATIVENESS:
                    sentiment = -1
            if lemma in CHANGE_IN_THE_LEXICON.keys():
                sentiment = CHANGE_IN_THE_LEXICON[lemma]
            if lemma == 'not':
                sentiment = 0.0
            indexes = word_index[lemma]
            updated_train_word_score[indexes] = sentiment
            #temp.append(sentiment)
        else:
            continue
        #if all(p == 0.0 for p in temp):
            #print(1)
        #else: 
            #continue
        
    updated_real_train.append(updated_train_word_score) 

updated_classifier_liblinear = svm.LinearSVC()
updated_classifier_liblinear.fit(updated_real_train, train_labels)
change_in_the_lexicon = {}

for marcia in riprova_tutto.keys():
        for h in range(0,10):
            print('check if ', marcia, 'was changed in the right way:')
            new_score = CHANGE_IN_THE_LEXICON[marcia] + uniform(-1,1)
            performance_baseline = 0.0
            right = 0 
            wrong = 0
            performance = 0.0
                    #change the score of the word
            for ll in range(len(test_data)):
                check_word_in_sentence = []
                tagged_sentence = pos_tag(test_data[ll])
                for word, tag in tagged_sentence:
                    wn_tag = penn_to_wn(tag)
                    if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV, wn.VERB):
                        continue
                    lemma = lemmatizer.lemmatize(word, pos=wn_tag)
                    if not lemma:
                        continue
                    check_word_in_sentence.append(lemma)
                if marcia in check_word_in_sentence:
                            #print('ce un altro errore')
                    compare = test_labels[ll]
                    new_test_word_score = np.zeros(VOCABULARY_SIZE)
                    for yy in check_word_in_sentence:
                        if yy in list_new:
                            sentiment = 0.0
                            if yy == marcia:
                                sentiment = new_score
                            else:
                                if yy in new_sentiment.keys():
                                    sentiment = new_sentiment[yy]
                                else:
                                    if yy in POSITIVENESS:
                                        sentiment = 1
                                    elif yy in NEGATIVENESS:
                                        sentiment = -1
                                if yy in CHANGE_IN_THE_LEXICON.keys():
                                    sentiment = CHANGE_IN_THE_LEXICON[yy]
                            if yy == 'not':
                                sentiment = 0.0
                            indexes = word_index[yy]
                            new_test_word_score[indexes] = sentiment
                        else:
                            continue
                    prova = []
                    prova.append(new_test_word_score)
                    updated_prediction_liblinear = updated_classifier_liblinear.predict(prova)
                    if 'not' in check_word_in_sentence:
                        if updated_prediction_liblinear == 'positive':
                            updated_prediction_liblinear = 'negative'
                        else:
                            updated_prediction_liblinear = 'positive'
                            
                    if compare == updated_prediction_liblinear:
                        right += 1
                    else:
                        wrong += 1    
                else:
                    continue
            performance = (right/(right+wrong))
            #print('new perf %f' % (performance))
                    #check if there is an improvement
            if performance > riprova_tutto[marcia]:
                word_with_new_score = marcia
                print("was not ok, so PS''' = ", new_score)
                change_in_the_lexicon[word_with_new_score] = new_score #cosi cosnidero i cambiamenti solo da un tweet all'altro e non per ogni parola
                break
            else:
                continue

CHANGE_IN_THE_LEXICON.update(change_in_the_lexicon)
print('after reinforcement and convergence: ', CHANGE_IN_THE_LEXICON)

end_real_train = []
end_real_test = []        
#train again SVM with new scores given by the updated sentiment lexicon final_train_word_score
for j in range(len(train_data)):
    end_train_word_score = np.zeros(VOCABULARY_SIZE)
    tagged_sentence = pos_tag(train_data[j])
    for word, tag in tagged_sentence:
        wn_tag = penn_to_wn(tag)
        if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV, wn.VERB):
            continue

        lemma = lemmatizer.lemmatize(word, pos=wn_tag)
        if not lemma:
            continue

        if lemma in list_new: 
        # Take the first sense, the most common
            sentiment = 0.0   
            if lemma in new_sentiment.keys():
                sentiment = new_sentiment[lemma]
            else:
                if lemma in POSITIVENESS:
                    sentiment = 1
                elif lemma in NEGATIVENESS:
                    sentiment = -1
            if lemma in CHANGE_IN_THE_LEXICON.keys():
                sentiment = CHANGE_IN_THE_LEXICON[lemma]
            if lemma == 'not':
                sentiment = 0.0
            indexes = word_index[lemma]
            end_train_word_score[indexes] = sentiment
            #temp.append(sentiment)
        else:
            continue

        
    end_real_train.append(end_train_word_score)

for l in range(len(test_data)):      
    end_test_word_score = np.zeros(VOCABULARY_SIZE)
    #test_word_score = np.zeros(VOCABULARY_SIZE)
    tagged_sentence = pos_tag(test_data[l])
    for word, tag in tagged_sentence:
        wn_tag = penn_to_wn(tag)
        if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV, wn.VERB):
            continue

        lemma = lemmatizer.lemmatize(word, pos=wn_tag)
        if not lemma:
            continue

        if lemma in list_new: 
        # Take the first sense, the most common
            sentiment = 0.0   
            if lemma in new_sentiment.keys():
                sentiment = new_sentiment[lemma]
            else:
                if lemma in POSITIVENESS:
                    sentiment = 1
                elif lemma in NEGATIVENESS:
                    sentiment = -1
            if lemma in CHANGE_IN_THE_LEXICON.keys():
                sentiment = CHANGE_IN_THE_LEXICON[lemma]
            if lemma == 'not':
                sentiment = 0.0
            indexes = word_index[lemma]
            end_test_word_score[indexes] = sentiment             
        else:
            continue
    end_real_test.append(end_test_word_score)


# Perform classification with SVM, kernel=linear
end_classifier_liblinear = svm.LinearSVC()
end_classifier_liblinear.fit(end_real_train, train_labels)
ok = 0.000
no = 0.000
for k in range(len(test_data)):
    controlla_not = []
    tagged_sentence = pos_tag(test_data[k])
    for word, tag in tagged_sentence:
        wn_tag = penn_to_wn(tag)
        if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV, wn.VERB):
            continue

        lemma = lemmatizer.lemmatize(word, pos=wn_tag)
        if not lemma:
            continue
        controlla_not.append(lemma)
        
    prediction = end_classifier_liblinear.predict(end_real_test[k])
    if 'not' in controlla_not:
        if prediction == 'positive':
            prediction = 'negative'
        else:
            prediction = 'positive'
            
    if prediction == test_labels[k]:
        ok += 1.0
    else:
        no += 1.0
accuracy = (ok/no)*100
print('accuracy_with_reinforcement: ', accuracy)